## Read in everything

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
from math import ceil
import numpy as np
pd.options.display.max_columns = 1000
from google.colab import files
import urllib.request
from zipfile import ZipFile
from io import BytesIO

In [0]:
proc_zip_url = 'https://github.com/fielddaylab/opengamedata/blob/master/jupyter/lakeland_data/LAKELAND_20191201_to_20191231_b2cf46d_proc.zip?raw=true'
resp = urllib.request.urlopen(proc_zip_url)
zipfile = ZipFile(BytesIO(resp.read()))
with zipfile.open(zipfile.namelist()[0]) as f:
    df = pd.read_csv(f)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1599) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df.head()

,lvl0_EventCount,lvl1_EventCount,lvl2_EventCount,lvl3_EventCount,lvl4_EventCount,lvl5_EventCount,lvl6_EventCount,lvl7_EventCount,lvl8_EventCount,lvl9_EventCount,lvl0_ActiveEventCount,lvl1_ActiveEventCount,lvl2_ActiveEventCount,lvl3_ActiveEventCount,lvl4_ActiveEventCount,lvl5_ActiveEventCount,lvl6_ActiveEventCount,lvl7_ActiveEventCount,lvl8_ActiveEventCount,lvl9_ActiveEventCount,lvl0_durationInSecs,lvl1_durationInSecs,lvl2_durationInSecs,lvl3_durationInSecs,lvl4_durationInSecs,lvl5_durationInSecs,lvl6_durationInSecs,lvl7_durationInSecs,lvl8_durationInSecs,lvl9_durationInSecs,lvl0_count_gamestate_logs,lvl1_count_gamestate_logs,lvl2_count_gamestate_logs,lvl3_count_gamestate_logs,lvl4_count_gamestate_logs,lvl5_count_gamestate_logs,lvl6_count_gamestate_logs,lvl7_count_gamestate_logs,lvl8_count_gamestate_logs,lvl9_count_gamestate_logs,lvl0_count_achievements,lvl1_count_achievements,lvl2_count_achievements,lvl3_count_achievements,lvl4_count_achievements,lvl5_count_achievements,lvl6_count_achievements,lvl7_count_achievements,lvl8_count_achievements,lvl9_count_achievements,lvl0_count_buy_home,lvl1_count_buy_home,lvl2_count_buy_home,lvl3_count_buy_home,lvl4_count_buy_home,lvl5_count_buy_home,lvl6_count_buy_home,lvl7_count_buy_home,lvl8_count_buy_home,lvl9_count_buy_home,lvl0_count_buy_food,lvl1_count_buy_food,lvl2_count_buy_food,lvl3_count_buy_food,lvl4_count_buy_food,lvl5_count_buy_food,lvl6_count_buy_food,lvl7_count_buy_food,lvl8_count_buy_food,lvl9_count_buy_food,lvl0_count_buy_farm,lvl1_count_buy_farm,lvl2_count_buy_farm,lvl3_count_buy_farm,lvl4_count_buy_farm,lvl5_count_buy_farm,lvl6_count_buy_farm,lvl7_count_buy_farm,lvl8_count_buy_farm,lvl9_count_buy_farm,lvl0_count_buy_fertilizer,lvl1_count_buy_fertilizer,lvl2_count_buy_fertilizer,lvl3_count_buy_fertilizer,lvl4_count_buy_fertilizer,lvl5_count_buy_fertilizer,lvl6_count_buy_fertilizer,lvl7_count_buy_fertilizer,lvl8_count_buy_fertilizer,lvl9_count_buy_fertilizer,lvl0_count_buy_livestock,lvl1_count_buy_livestock,lvl2_count_buy_livestock,lvl3_count_buy_livestock,lvl4_count_buy_livestock,lvl5_count_buy_livestock,lvl6_count_buy_livestock,lvl7_count_buy_livestock,lvl8_count_buy_livestock,lvl9_count_buy_livestock,lvl0_count_buy_skimmer,lvl1_count_buy_skimmer,lvl2_count_buy_skimmer,lvl3_count_buy_skimmer,lvl4_count_buy_skimmer,lvl5_count_buy_skimmer,lvl6_count_buy_skimmer,lvl7_count_buy_skimmer,lvl8_count_buy_skimmer,lvl9_count_buy_skimmer,lvl0_count_buy_sign,lvl1_count_buy_sign,lvl2_count_buy_sign,lvl3_count_buy_sign,lvl4_count_buy_sign,lvl5_count_buy_sign,lvl6_count_buy_sign,lvl7_count_buy_sign,lvl8_count_buy_sign,lvl9_count_buy_sign,lvl0_count_buy_road,lvl1_count_buy_road,lvl2_count_buy_road,lvl3_count_buy_road,lvl4_count_buy_road,lvl5_count_buy_road,lvl6_count_buy_road,lvl7_count_buy_road,lvl8_count_buy_road,lvl9_count_buy_road,lvl0_money_spent,lvl1_money_spent,lvl2_money_spent,lvl3_money_spent,lvl4_money_spent,lvl5_money_spent,lvl6_money_spent,lvl7_money_spent,lvl8_money_spent,lvl9_money_spent,lvl0_money_spent_home,lvl1_money_spent_home,lvl2_money_spent_home,lvl3_money_spent_home,lvl4_money_spent_home,lvl5_money_spent_home,lvl6_money_spent_home,lvl7_money_spent_home,lvl8_money_spent_home,lvl9_money_spent_home,lvl0_money_spent_food,lvl1_money_spent_food,lvl2_money_spent_food,lvl3_money_spent_food,lvl4_money_spent_food,lvl5_money_spent_food,lvl6_money_spent_food,lvl7_money_spent_food,lvl8_money_spent_food,lvl9_money_spent_food,lvl0_money_spent_farm,lvl1_money_spent_farm,lvl2_money_spent_farm,lvl3_money_spent_farm,lvl4_money_spent_farm,lvl5_money_spent_farm,lvl6_money_spent_farm,lvl7_money_spent_farm,lvl8_money_spent_farm,lvl9_money_spent_farm,lvl0_money_spent_fertilizer,lvl1_money_spent_fertilizer,lvl2_money_spent_fertilizer,lvl3_money_spent_fertilizer,lvl4_money_spent_fertilizer,lvl5_money_spent_fertilizer,lvl6_money_spent_fertilizer,lvl7_money_spent_fertilizer,lvl8_money_spent_fertilizer,lvl9_money_spent_fertilizer,lvl0_money_spent_livestock,lvl1_money_spent_livestock,lvl2_money_spen

## Filter out which columns I want to use.
#### Print all column names and indices available

In [0]:
print(*[(i,c) for i,c in enumerate(df.columns)])

#### Decided to use the following, as well as the averages of the seconds_per_blurb features :

In [0]:
raw_events_to_take = [0,10,70,80, 1589, 1595, 1594,1581,1582, #1487,1488,1489, #1579,1580,1562
                      1563, 1370, 1374,1375,1376,1377,1378,1379,1380,1381, 1577,1573,1480,1598]
avgs_to_take = [range(1442,1468)]
print(*["- "+df.columns[i]+f" ({i})" for i in sorted(raw_events_to_take)],sep='\n')

## Filtering
- Calculate avg_secs_per_blurb
- Filtered only the columns I was interested in.
- Filtered out the sessions that used SPYPARTY (debug=1)
- Filtered any sessions that were not between 60 seconds and 7200 seconds
- Filtered out continues
- Filtered out any sessions that did not have at least 3 active events in lvl0 and 10 in the session
- Filled any NaNs with 0s (Any NaN in count_buy in levels means 0. That is can be fixed but I just thought it was low priority)

In [0]:
def avg_columns(df, col_list):
    return df.iloc[:,col_list].mean(axis=1)
df2 = df.iloc[:,raw_events_to_take].copy()
avg_secs_per_blurb = df.iloc[:, list(avgs_to_take[0])].replace(0,np.nan).mean(axis=1, skipna=True)
df2['avg_secs_per_blurb'] = avg_secs_per_blurb
df2 = df2[df2['debug'] != 1]
df2 = df2[df2['debug'] < 1]
df2 = df2[(60 < df2['sessDuration']) & (df2['sessDuration'] < 60*60*1)]
df2 = df2[df2['lvl0_ActiveEventCount'] > 2]
df2 = df2[df2['sess_ActiveEventCount'] > 9]
df2 = df2.drop(columns=['debug'])
df2.fillna(0,inplace=True)
df2

#### Descriptive stats

In [0]:
df2.describe()

#### Histograms for each selected feature (after filtering, before preprocessing)

In [0]:
df2.hist(figsize=(20,20),bins=50)

## Preprocessing Approach 1: RobustScaler

In [0]:
from sklearn import preprocessing
scaler = preprocessing.RobustScaler().fit(df2.values)
scaled_df = pd.DataFrame(scaler.transform(df2.values),columns=df2.columns)
scaled_df.hist(figsize=(20,20),bins=np.linspace(-10,10))

In [0]:
scaled_df.describe()

#### Normalize.
But as you can see, the graphs aren't too normal still. I want to try to do the log transform.

In [0]:
normalized_df = pd.DataFrame(preprocessing.normalize(scaled_df),columns=df2.columns)
normalized_df.hist(figsize=(20,20), bins=50)

#### Did PCA on this, then KMeans.
Realized that even though I removed sessions with the large sessDurations, there are still sessions with for example 615 stdevs from the mean sess_money_earned. That is an issue not solved by standard scaling.

In [0]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans, DBSCAN
pca = PCA(n_components=3)
projected = pca.fit_transform(normalized_df.values)
plt.figure(figsize=(20,20))
plt.scatter(projected[:,0], projected[:,1], c=KMeans(5).fit(normalized_df.values).labels_)

In [0]:
from mpl_toolkits.mplot3d import Axes3D
plt.figure(figsize=(20,20))
ax = plt.axes(projection='3d')
ax.scatter3D(projected[:,0], projected[:,1], projected[:,2], c=KMeans(5).fit(normalized_df.values).labels_);

## Preprocessing approach 3 -- log transform
I added 1 to every value to get rid of the 0 errors in the logs. Maybe that was a bad idea, but I assumed that there would be very few values between 0 and 1 that would lose a lot of information because of this. I can try again another day with .1 or something?

In [0]:
from math import log
logged_df = pd.DataFrame(np.log(df2.to_numpy()+1),columns=df2.columns)
logged_df.hist(figsize=(20,20))

#### At first I was worried that these didn't look very normal so I added a scaler WHICH NOW I REALIZE WAS NOT HELPING

In [0]:
from sklearn import preprocessing
scaler = preprocessing.RobustScaler().fit(logged_df.values)
scaled_df = pd.DataFrame(scaler.transform(logged_df.values),columns=df2.columns)

In [0]:
scaled_df.hist(figsize=(20,20))

#### Add normalization step

In [0]:
normalized_log_df = pd.DataFrame(preprocessing.normalize(scaled_df),columns=df2.columns)

In [0]:
projected = pca.fit_transform(normalized_log_df.values)
plt.figure(figsize=(20,20))
plt.scatter(projected[:,0], projected[:,1], c=KMeans(5).fit(normalized_log_df.values).labels_)

In [0]:
from mpl_toolkits.mplot3d import Axes3D
plt.figure(figsize=(20,20))
ax = plt.axes(projection='3d')
ax.scatter3D(projected[:,0], projected[:,1], projected[:,2], c=KMeans(5).fit(normalized_log_df.values).labels_);

In [0]:
pca.fit_transform(normalized_log_df.values).shape

In [0]:
def get_cluster_stats(df,labels):
    tdf = df.copy()
    tdf["label1"] = labels
    cluster_df = pd.DataFrame()
    for c in range(5):
        cluster_df[(c,"mean")] = tdf[tdf['label1']==c].mean()
        cluster_df[(c,"stdev")] = tdf[tdf['label1']==c].std()
    return cluster_df

In [0]:
get_cluster_stats(df2,KMeans(5).fit(normalized_log_df.values).labels_)

In [0]:
get_cluster_stats(df2,KMeans(5).fit(normalized_df.values).labels_)

## Stuff below here is all stuff I did before I realized that the scaling functions were not actually normalizing the data

In [0]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans,DBSCAN
from hdbscan import HDBSCAN

In [0]:
projection = TSNE().fit_transform(df2.values)

In [0]:
labels = HDBSCAN(20).fit(df2.values).labels_
plt.scatter(*projection.T, c=labels)

In [0]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=2)
X = df2.values
nbrs = neigh.fit(X)
distances, indices = nbrs.kneighbors(X)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
axes = plt.gca()
axes.set_ylim([0,1000])
plt.plot(distances)

In [0]:
df2['label1'] = labels

In [0]:
df2

In [0]:
features = list(df2.columns)
def scatter_with_labels(x_index,y_index,label_index):
    xy_names = (features[x_index], features[y_index])
    X = df2[[features[x_index], features[y_index], features[label_index]]].dropna().to_numpy()
    plt.scatter(X[:, 0], X[:, 1], c=X[:,2], cmap='viridis')
    xlabel, ylabel = xy_names
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [0]:
{i:f for i,f in enumerate(features)}

In [0]:

cluster_df = pd.DataFrame()
for c in range(50):
    cluster_df[(c,"mean")] = df2[df2['label1']==c].mean()
    cluster_df[(c,"stdev")] = df2[df2['label1']==c].std()
cluster_df

In [0]:
df2[df2["label1"]==2].describe()

In [0]:
df2[df2["label1"]==3].mean()

In [0]:
get_cluster_data(df2, 50, "label1")

## Preprocessing Approach 2: PowerTransformer
I had to use the yeo-johnson instead of box-cox because of so many 0 values in the matrix. Still did not yield a good result with kmeans, although there does seem to be an area of high density that can be picked up with a density based metric like DBSCAN etc

In [0]:
pipeline = make_pipeline(preprocessing.PowerTransformer(method='yeo-johnson'),PCA(n_components=2))
projected = pipeline.fit_transform(df2.values)
plt.figure(figsize=(20,20))
plt.scatter(projected[:,0], projected[:,1],c=KMeans(4).fit(df2.values).labels_)

#### Find input epsilon for DBSCAN
The x_value at the point of highest change of slope in the nearest neighbor plot is the input epilon for DBSCAN. So run this plot, put in 100 for DBSCAN

In [0]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=2)
X = df2.values
nbrs = neigh.fit(X)
distances, indices = nbrs.kneighbors(X)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
axes = plt.gca()
axes.set_ylim([0,1000])
plt.plot(distances)

In [0]:
for e in range(100,250,50):
    plt.figure(figsize=(20,20))
    plt.scatter(projected[:,0], projected[:,1],c=DBSCAN(e).fit(df2.values).labels_)
    print(f"Epsilon = {e}")
    plt.show()

#### Look at stats of the yeo-johnson transform.
I wanted to see if theres obvious point of confusion there, but I did not see anything that jumped out at me. I wonder if perhaps I normalized too much and left out some other point of distance. In rapidminer, it easily spits out quick differences between clusters, but I can't exactly tell if there are any cluster differences here that I am looking for.


In [0]:
pd.DataFrame(preprocessing.PowerTransformer(method='yeo-johnson').fit_transform(df2.values),columns=df2.columns).describe()

In [0]:
import matplotlib
print(matplotlib.__version__)